# The Stanford Sentiment Treebank 
The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. We use the two-way (positive/negative) class split, and use only sentence-level labels.

In [1]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export BUCKET_TRANSLATION_NAME=your_gcp_gs_bucket_translation_name`  
`export BUCKET_STAGING_NAME=your_gcp_gs_bucket_staging_name` 
`export REGION=your_region`  
`export PATH_SAVE_MODEL=your_path_to_save_model`  
`export CLOUDSDK_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  
`export CLOUDSDK_GSUTIL_PYTHON=your_path/conda-env/env_gcp_sdk/bin/python`  

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    # [EDIT ME]
    export DIR_PROJ=your_path_git_repository
    export PYTHONPATH=$DIR_PROJ/src
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1='\[\e[91m\]\u@:\[\e[32m\]\w\[\e[0m\]$'
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import Packages

In [1]:
import tensorflow as tf
import tensorflow_datasets

from tensorflow.keras.utils import to_categorical

from transformers import (
    BertConfig,
    BertTokenizer,
    XLMRobertaTokenizer,
    TFBertModel,
    TFXLMRobertaModel,
    TFBertForSequenceClassification,
    glue_convert_examples_to_features,
    glue_processors
)

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

from google.cloud import storage

import math
import numpy as np
import os
import glob
import time
from datetime import timedelta
import shutil
from datetime import datetime
import pickle
import re
import codecs
import json
from googleapiclient import discovery
from googleapiclient import errors

## Check configuration

In [2]:
print(tf.version.GIT_VERSION, tf.version.VERSION)

v2.2.0-rc4-8-g2b96f3662b 2.2.0


In [3]:
print(tf.keras.__version__)

2.3.0-tf


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)>0:
    for gpu in gpus:
        print('Name:', gpu.name, '  Type:', gpu.device_type)
else:
    print('No GPU available !!!!')

No GPU available !!!!


## Define Paths

In [5]:
try:
    data_dir=os.environ['PATH_DATASETS']
except KeyError:
    print('missing PATH_DATASETS')
try:   
    tensorboard_dir=os.environ['PATH_TENSORBOARD']
except KeyError:
    print('missing PATH_TENSORBOARD')
try:   
    savemodel_dir=os.environ['PATH_SAVE_MODEL']
except KeyError:
    print('missing PATH_SAVE_MODEL')

## Import local packages

In [7]:
import preprocessing.preprocessing as pp
import utils.model_metrics as mm
import utils.model_utils as mu

/Users/tarrade/anaconda-release/conda-env/env_multilingual_class/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
import importlib
importlib.reload(pp);
importlib.reload(mm);
importlib.reload(mu);

## Check the local model

In [9]:
savemodel_path = os.path.join(savemodel_dir, 'saved_model')
os.makedirs(savemodel_path, exist_ok=True)

In [10]:
model=tf.keras.models.load_model(os.path.join(savemodel_path, 'tf_bert_classification'))

In [11]:
# check the saved model
print('Model: {}'.format(model.name))
for i in os.listdir(os.path.join(savemodel_path,model.name)):
        print(' ',i)

Model: tf_bert_classification
  projector_config.pbtxt
  variables
  history
  history_per_step
  train
  saved_model.pb
  assets
  validation


In [12]:
model.summary()

Model: "tf_bert_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  167356416 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 167,357,954
Trainable params: 167,357,954
Non-trainable params: 0
_________________________________________________________________


In [13]:
source_bucket_name = savemodel_path+'/'+model.name

In [14]:
for root, dirs, files in os.walk(source_bucket_name):
    for name in files:
        if not 'history' in name:
            print(os.path.join('.../', name))

.../projector_config.pbtxt
.../saved_model.pb
.../variables.data-00000-of-00001
.../variables.index
.../keras_embedding.ckpt-1.index
.../checkpoint
.../keras_embedding.ckpt-0.index
.../events.out.tfevents.1587997306.Fabien-Tarrades-MacBook-Pro.local.20696.11162.v2
.../events.out.tfevents.1587997350.Fabien-Tarrades-MacBook-Pro.local.profile-empty
.../events.out.tfevents.1587999214.Fabien-Tarrades-MacBook-Pro.local.22120.11162.v2
.../keras_embedding.ckpt-1.data-00000-of-00001
.../keras_embedding.ckpt-0.data-00000-of-00001
.../local.trace
.../local.trace
.../events.out.tfevents.1587999352.Fabien-Tarrades-MacBook-Pro.local.22120.34079.v2
.../events.out.tfevents.1587997456.Fabien-Tarrades-MacBook-Pro.local.20696.34079.v2


The **variables** directory contains a standard training checkpoint (see the guide to training checkpoints).  
The **assets** directory contains files used by the TensorFlow graph, for example text files used to initialize vocabulary tables.  
The **saved_model.pb** file stores the actual TensorFlow program, or model, and a set of named signatures, each identifying a function that accepts tensor inputs and produces tensor outputs.

In [15]:
os.environ['MODEL_LOCAL']=savemodel_path+'/'+model.name

SavedModels may contain multiple variants of the model (multiple v1.MetaGraphDefs, identified with the **--tag_set** flag to saved_model_cli), but this is rare. APIs which create multiple variants of a model include 

In [42]:
%%bash
saved_model_cli show --dir $MODEL_LOCAL --tag_set serve 

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [43]:
%%bash
saved_model_cli show --dir $MODEL_LOCAL --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['attention_mask'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 128)
      name: serving_default_attention_mask:0
  inputs['input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 128)
      name: serving_default_input_ids:0
  inputs['token_type_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 128)
      name: serving_default_token_type_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


## Copy the local model on GCP

In [67]:
destination_bucket_name = 'saved_model/tf_bert_classification'
copy_model_gcp=False

In [68]:
# will take some time since the size of the model is 2 GB!
buket_name = os.environ['BUCKET_NAME']
if copy_model_gcp:
    mu.copy_local_directory_to_gcs(source_bucket_name, buket_name, destination_bucket_name)

In [133]:
os.environ['MODEL_GCP']='gs://'+os.environ['BUCKET_NAME']+'/saved_model/'+model.name

In [135]:
%%bash
saved_model_cli show --dir $MODEL_GCP --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['attention_mask'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 128)
      name: serving_default_attention_mask:0
  inputs['input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 128)
      name: serving_default_input_ids:0
  inputs['token_type_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 128)
      name: serving_default_token_type_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 2)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


## Model serving setup

In [1]:
project_name = os.environ['PROJECT_ID']
project_id = 'projects/{}'.format(project_name)
ai_platform_serving = discovery.build('ml', 'v1')

NameError: name 'os' is not defined

### Check models already deployed

In [72]:
request = ai_platform_serving.projects().models().list(parent=project_id)

In [74]:
# Make the call.
try:
    response = request.execute()
    print('List of model:')
    if 'models' in response.keys():
        for i in response['models']:
            print('  Model \'s name: {}:'.format(i['name'].split('/')[-1]))
            print('    descrition: {}'.format(i['description']))
            print('    regions: {}'.format(i['regions']))

except errors.HttpError as err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

List of model:


### Create a new model

In [75]:
# defining the name of the model for online prediction
name_model = model.name+'_test'
description_model = 'this is a model for test'

# Normal VM has a model size of 500 MB for more you need to use a specific n1-standard-2 VM (2 GB) for online prediction. It is only available in us-central1.
region_model = 'us-central1'

# Create a dictionary with the fields from the request body.
request_dict = {'name': name_model,
                'description': description_model}

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().models().create(parent=project_id, 
                                                         body=request_dict)

In [76]:
# Make the call.
try:
    response = request.execute()
    print('Name of the model: {}:'.format(response['name'].split('/')[-1]))
    print('  descrition: {}'.format(response['description']))
    print('  regions: {}'.format(response['regions']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

Name of the model: tf_bert_classification_test:
  descrition: this is a model for test
  regions: ['us-central1']


### Defined all parameters and upload our models

In [77]:
# defining the name of the model for online prediction
name_model = model.name+'_test'
description_model = ' this is a model for test'
parentId = 'projects/{}/models/{}'.format(project_name, name_model)
# Normal VM has a model size of 500 MB for more you need to use a specific n1-standard-2 VM (2 GB) for online prediction. It is only available in us-central1.
region_model = 'us-central1'

model_binaries = 'gs://'+os.environ['BUCKET_NAME']+'/saved_model/'+model.name
version = 'V1'

# Create a dictionary with the fields from the request body.
request_dict = {'machineType': 'n1-standard-2',
                'runtimeVersion': '2.1',
                'pythonVersion': '3.7',
                'description': 'This is sentiment classifier using BERT and fine tune on SST-2 for test',
                'deploymentUri': model_binaries,
                'name': version
               }

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().models().versions().create(parent=parentId, 
                                                                    body=request_dict)

In [78]:
# Make the call.
try:
    response = request.execute()
    print('Name of the model: {}:'.format(response['name'].split('/')[-1]))
    print('  descrition: {}'.format(response['metadata']['version']['description']))
    print('  runtimeVersion: {}'.format(response['metadata']['version']['runtimeVersion']))
    print('  framework: {}'.format(response['metadata']['version']['framework']))
    print('  machineType: {}'.format(response['metadata']['version']['machineType']))
    print('  pythonVersion: {}'.format(response['metadata']['version']['pythonVersion']))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

Name of the model: create_tf_bert_classification_test_V1-1589892973851:
  descrition: This is sentiment classifier using BERT and fine tune on SST-2 for test
  runtimeVersion: 2.1
  framework: TENSORFLOW
  machineType: n1-standard-2
  pythonVersion: 3.7


### Check that the new modelal was deployed

In [97]:
request = ai_platform_serving.projects().models().list(parent=project_id)

In [98]:
# Make the call.
try:
    response = request.execute()
    print('List of model:')
    for i in response['models']:
        print('  Model \'s name: {}:'.format(i['name'].split('/')[-1]))
        print('    descrition: {}'.format(i['description']))
        print('    regions: {}'.format(i['regions']))

except errors.HttpError as err:
    # Something went wrong, print out some information.
    print('There was an error creating the model. Check the details:')
    print(err._get_reason())

List of model:
  Model 's name: tf_bert_classification_test:
    descrition: this is a model for test
    regions: ['us-central1']


## Model serving inference

### Prepare data for online prediction

example of format:

```
{'instances': 
  [
    {'input_ids': [101, 143, 18267, 15470, 90395, ...], 
     'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, .....], 
     'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, .....]
     }, 
     {'input_ids': [101, 17664, 143, 30728, .........], 
      'attention_mask': [1, 1, 1, 1, 1, 1, 1, .......], 
      'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, ....]
      }
  ]
}

```

In [54]:
# load data
tfrecord_data_dir=data_dir+'/tfrecord/sst2/tf_bert_classification/valid'
os.makedirs(tfrecord_data_dir, exist_ok=True)
valid_files = tf.data.TFRecordDataset(tfrecord_data_dir+'/valid_dataset.tfrecord')
valid_dataset = valid_files.map(pp.parse_tfrecord_glue_files)

In [128]:
# format data for online prediction
#serving_data_dir=data_dir+'/serving/sst2'
#os.makedirs(serving_data_dir, exist_ok=True)
#json_file = serving_data_dir+'/input_predict_gcloud.json' 
np_array = np.array(list(valid_dataset.as_numpy_iterator()))

data_prediction=[]

with codecs.open(json_file, 'w', encoding='utf-8') as f:
    for el in np_array[0:10]:
        instance={'input_ids': el[0]['input_ids'].tolist(), 'attention_mask': el[0]['attention_mask'].tolist(), 'token_type_ids': el[0]['token_type_ids'].tolist()}
        #instance2=[el[0]['input_ids'].tolist(), el[0]['attention_mask'].tolist(), el[0]['token_type_ids'].tolist()]
        data_prediction.append(instance)

In [137]:
np.shape(data_prediction)

(10,)

In [138]:
print(data_prediction[0:2])

[{'input_ids': [101, 143, 18267, 15470, 90395, 10453, 10202, 14985, 10114, 11061, 16722, 10533, 20448, 83617, 10151, 10103, 15252, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### Make online prediction

In [131]:
# model
version_name='V1'
model_model='tf_bert_classification_test'
parent = 'projects/{}/models/{}/versions/{}'.format(project_name, model_model, version_name)

# data for prediction
request_data = {"instances": data_prediction}

# Create a request to call projects.models.create.
request = ai_platform_serving.projects().predict(body=request_data, 
                                                 name=parent)

In [132]:
# Make the call.
try:
    response = request.execute()
    print('predictions:')
    for i in response['predictions']:
        print('  {}'.format(i))

except errors.HttpError as err:
    # Something went wrong, print out soFinme information.
    print('There was an error making prediction. Check the details:')
    print(err._get_reason())

predictions:
  [-0.0537839085, 0.0720201954]
  [-0.0632888377, 0.0719291791]
  [-0.0953264311, 0.0387066]
  [-0.0534419641, 0.058990825]
  [-0.0121499831, 0.0440233871]
  [-0.262184769, 0.110144384]
  [-0.204168573, 0.0657891855]
  [-0.0427531525, 0.0580056831]
  [-0.070497252, 0.0428885221]
  [0.0480646119, 0.0557359233]


### Make prediction using local model to test the input data

In [139]:
# not possible ? probably not very useful